#### Homework - Spark Programming on Taxicab Report Dataset 

The purpose of this exercise is to write some `pyspark` code that does some computation over a large dataset. Specifically, your Spark program will analyze a dataset consisting of New York City Taxi trip reports in the Year 2013. The dataset was released under the FOIL (The Freedom of Information Law) and made public by Chris Whong (https://chriswhong.com/open-data/foiling-nycs-boro-taxi-trip-data/).

The dataset is a simple `csv` file. Each taxi trip report is a different line in the file. Among
other things, each trip report includes the starting point, the drop-off point, corresponding timestamps, and
information related to the payment. The data are reported by the time that the trip ended, i.e., upon arrive in
the order of the drop-off timestamps.
The attributes present on each line of the file are, in order:

| attribute    | description                                                       |
| -------------|-------------------------------------------------------------------|
| medallion    | an md5sum of the identifier of the taxi - vehicle bound (Taxi ID) |
| hack_license | an md5sum of the identifier for the taxi license (driver ID)      |
| vendor_id    |identifies the vendor  |
| pickup_datetime	|time when the passenger(s) were picked up  |
| payment_type	 |the payment method -credit card or cash  |
| fare_amount	 |fare amount in dollars  |
| surcharge	 |surcharge in dollars  |
| mta_tax	 |tax in dollars  |
| tip_amount	 |tip in dollars  |
| tolls_amount	 |bridge and tunnel tolls in dollars  |
| total_amount	 |total paid amount in dollars  |

Data files:
* `taxi_small_subset.csv` - This is a subset of the entire big file. You can examine this file to see what the data look like. Also, you can use this file for running your code in a single-node platform (e.g., in Vocareum) and debug it, before running your code on the big file in the cluster.   
* `2013_weekdays.csv` - This is a file with the dates of 365 days of the year 2013 with their corresponding week day. This file is used in task 4 to do join.
* S3 URI `s3://comp643bucket/homework/spark_taxicab/trip*` - This is the address of the entire dataset available in S3, which is a big file (18.4 GB). Once you debugged your code on the small subset, your final task is to run your code on this big file over an EMR cluster in AWS.

**For this homework, you need to complete 5 tasks described below.** 

**For tasks 1 through 4, write your Spark code in this Jupyter Notebook and run your code on the small subset of data, i.e., `taxi_small_subset.csv`, in Vocareum. This helps you debug your Spark program easier since you're running it in an interactive single-node platform and on a small dataset.**     

**Once you've debugged your code on a small dataset, for task 5, you need to execute your Spark code for tasks 1 through 4, in an AWS EMR cluster on the big dataset that is stored in S3 (`s3://comp643bucket/homework/spark_taxicab/trip*`).** 

In [2]:
import pyspark

In [3]:
# pyspark works best with java8 
# set JAVA_HOME enviroment variable to java8 path 
%env JAVA_HOME = /usr/lib/jvm/java-8-openjdk-amd64

env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


In [4]:
sc = pyspark.SparkContext()

**Read the data file into an RDD**

In [5]:
taxi = sc.textFile('data/taxi_small_subset.csv')

In [6]:
taxi.count()

71153

In [7]:
taxi.take(10)

['medallion, hack_license, vendor_id, pickup_datetime, payment_type, fare_amount, surcharge, mta_tax, tip_amount, tolls_amount, total_amount',
 '7DD1D6A5E432ACBD68A734587B589B9B,EF3FD28F7D39F614BF68B51F0256B050,CMT,2013-08-28 06:53:33,CSH,12,0,0.5,0,0,12.5',
 'CEBDF34FE2DA2E9233B87C2E703004FF,D9EA31E70BE082F423D42860FD4BD240,CMT,NULL,CSH,7,1,0.5,0,0,8.5',
 'A6E8AD830F49F7B358D52419084D42A0,B1F1E21144EC5D9EC144AF9E4FBF320E,CMT,2013/08/29 12:59:08,CSH,6,0,0.5,0,0,6.5',
 'AA5DA32A45ABF4F64F3AC01A4AD00B0F,B0A352468D7D2FEDC1BFD85A74BEE0A9,CMT,2013-08-30,CSH,11,0,0.5,0,0,11.5',
 '506DF1356EA4AAA9DE38EE3103CDAAF9,EBE5414C3D3E4DC06885874F0022917A,CMT,2013-08-27 17:26:48,CSH,$17.5,1,0.5,0,5.33,24.33',
 '801D1D2678B78ABC6F4FDBF224108FAB,FBF69C3FCBC582B89CF9646BEC45A724,CMT,2013-08-31 02:32:14,CSH,5.5,0.5,0.5,0,0,$6.5',
 'B07F45A08DAF5ED6AD1765DCF710B288,85998406637716860BE076B195318E14,CMT,2013.08.31 00:39:55,CSH,7.5,0.5,0.5,0,0,8.5',
 '4EE8A541D108B461356A596EEDEFD5E0,416BF2C641CCA4881C3FEB6D09

## Task 1 - clean the dataset (20 pts)

Write a Spark program that reads the dataset into an RDD, splits each line by `,` to extract field values, and cleans the RDD through the following steps:
* Remove lines with any missing value indicated by `NULL` 
* Validate the type of the following fields and remove lines with any invalid field value:
    * `pickup_datetime` must match this pattern 'YYYY-MM-DD HH:MM:SS'
    * All fileds in dollars (`fare_amount`, `surcharge`, `mta_tax`, `tip_amount`, `tolls_amount`, `total_amount`) must be non-negative numbers (with or without a decimal point)
    
After each step of cleaning, run `count()` on your RDD, to see how many lines have been left. 

Below, we give you a set of cells you can use to walk through the analysis procress. You are also welcome to simply write all of your code in one cell, following your own logic.

### Split each line by `,` to extract field values

In [8]:
cleaned = taxi.flatMap(lambda x: [x.split(',')])
cleaned.take(2)

[['medallion',
  ' hack_license',
  ' vendor_id',
  ' pickup_datetime',
  ' payment_type',
  ' fare_amount',
  ' surcharge',
  ' mta_tax',
  ' tip_amount',
  ' tolls_amount',
  ' total_amount'],
 ['7DD1D6A5E432ACBD68A734587B589B9B',
  'EF3FD28F7D39F614BF68B51F0256B050',
  'CMT',
  '2013-08-28 06:53:33',
  'CSH',
  '12',
  '0',
  '0.5',
  '0',
  '0',
  '12.5']]

### Clean the RDD

**Remove lines with any `NULL` value**

In [9]:
cleaned_no_nulls = cleaned.filter(lambda x: 'NULL' not in x)

**Run `count()` on your RDD to see how many lines have been left**

In [10]:
cleaned_no_nulls.count()

71141

**Remove lines with `pickup_datetime` that does not match this pattern 'YYYY-MM-DD HH:MM:SS'**

For this task, you can use Python `re` module along with your Spark code.

In [11]:
import re

In [12]:
pattern = re.compile(r"\d{4}-([0][1-9]|[1][012])-([0][1-9]|[12]\d|[3][01]) ([01]\d|[2][0-3]):([0-5]\d):([0-5]\d)")
cleaned_nulls_dates = cleaned_no_nulls.filter(lambda x: re.fullmatch(pattern, x[3]))

**Run `count()` on your RDD to see how many lines have been left**

In [13]:
cleaned_nulls_dates.count()

71134

**All the fields indicating an amount in dollar (`fare_amount`, `surcharge`, `mta_tax`, `tip_amount`, `tolls_amount`, `total_amount`) must be non-negative numeric (with or without decimal point) value. Remove lines with any value that does not match this pattern.** 

For this task, you can use Python `re` module along with your Spark code.

In [14]:
pattern2 = re.compile(r"\d+(\.\d\d?)?")
cleaned_nulls_dates_money = cleaned_nulls_dates.filter(lambda x: re.fullmatch(pattern2, x[5]) and re.fullmatch(pattern2, x[6])
                                                       and re.fullmatch(pattern2, x[7]) and re.fullmatch(pattern2, x[8])
                                                       and re.fullmatch(pattern2, x[9]) and re.fullmatch(pattern2, x[10]))

**Run `count()` on your RDD to see how many lines have been left**

In [15]:
cleaned_nulls_dates_money.count()

71128

## Task 2 - compute total revenue by dates (20 pts)

Write a Spark program on your derived cleaned RDD (from task 1) that computes the total amount of revenue (`total_amount` field) for each date (`pickup_datetime` field without time portions - only dates). Then, sort your RDD by the total revenue in ascending order and print out the 5 lines with the smallest total revenue. That shows the 5 dates with least total revenue.   

In [16]:
date_amount = cleaned_nulls_dates_money.map(lambda x: (x[3][:10],float(x[10])))
total_date_amount = date_amount.reduceByKey(lambda x, y: x + y)
total_date_amount_sorted = total_date_amount.sortBy(lambda x: x[1], ascending=True)

In [17]:
total_date_amount_sorted.take(5)

[('2013-01-23', 8.5),
 ('2013-08-10', 23.5),
 ('2013-04-07', 26.0),
 ('2013-03-24', 28.0),
 ('2013-03-21', 31.25)]

## Task 3 - compute total revenue by taxi drivers  (20 pts)

Write a Spark program on your derived cleaned RDD (from task 1) that computes the total amount of revenue (`total_amount` field) for each taxi driver (`hack_license`). Then, sort your RDD by the total revenue in descending order and print out the top 5 lines with the largest total revenue. That shows the 5 taxi drivers with most total revenue. 

In [18]:
driver_amount = cleaned_nulls_dates_money.map(lambda x: (x[1], float(x[10])))
total_driver_amount = driver_amount.reduceByKey(lambda x, y: x + y)
total_driver_amount_sorted = total_driver_amount.sortBy(lambda x: x[1], ascending=False)

In [19]:
total_driver_amount_sorted.take(5)

[('CFCD208495D565EF66E7DFF9F98764DA', 508.5),
 ('178C58D2C909125EE599C388CC1A311C', 356.9),
 ('83DDCD2CC7035BEBED7AC4255688308A', 355.0),
 ('B9E81BA07F0DDA5B2FBCA9B33CCC7C9A', 335.3),
 ('98949EA21D9A4DA151ADEE27E4DEDE7C', 333.32)]

## Task 4 - compute total revenue by weekday through join operation (20 pts)

Write a Spark program on your derived cleaned RDD (from task 1) that computes the total amount of revenue (`total_amount` field) for each 7 days of the week (Sunday through Saturday).

To extract the week days and experimenting more with Spark, we suggest that you use `join` RDD operation to join the taxi dataset with the provided `2013_weekdays.csv` file that contains the dates for 365 days of the year 2013 and their corresponding week days.    

First, read `2013_weekdays.csv` into an RDD, and split each line by `,` to extract the field values.

Then, manipulate this RDD and your derived cleaned RDD of taxi dataset (from task 1), to be able to join the two and compute the total revenue by weekday.  

Finally, sum the total amount per weekday, and return the result in descending order of the total revenue.

In [22]:
days = sc.textFile('data/2013_weekdays.csv')
cleaned_days = days.flatMap(lambda x: [x.split(',')]).map(lambda x: (x[0], x[1]))
joined_days = cleaned_days.join(total_date_amount)
joined_days_organized = joined_days.map(lambda x: (x[1][0], x[1][1]))
total_days_amount = joined_days_organized.reduceByKey(lambda x, y: x + y)
total_days_amount_sorted = total_days_amount.sortBy(lambda x: x[1], ascending=False)

In [24]:
total_days_amount_sorted.take(7)

[('Thursday', 161163.15),
 ('Saturday', 160515.67999999996),
 ('Friday', 160117.96),
 ('Tuesday', 155380.25),
 ('Wednesday', 154230.57),
 ('Monday', 138941.37),
 ('Sunday', 137148.43)]

## Task 5 - run on a big file in EMR cluster (20 pts)

For the last part of this homework, you need to run your Spark code for tasks 1 through 4, on a big file in S3, in an AWS EMR cluster. 

Follow the instructions on `Lab - Spark Intro (AWS)` to create and connect to an EMR cluster in AWS and run Spark programs in there. 

**For better efficiency, in the hardware configuration of your cluster, choose `m5.xlarge` as instance type, and type 4 as the number of instances.**  

The big file exists in this S3 URI: `s3://comp643bucket/homework/spark_taxicab/trip*.csv`

To read the big file from S3 into an RDD, use the code below:

`taxi = sc.textFile ("s3://comp643bucket/homework/spark_taxicab/trip*.csv")`

Repeat tasks 1 through 4 on this `taxi` RDD created from the big file, and print your results in the markdown cells below (keep the results from the small subset above). 

**Repeat task 1 on the big file in your EMR cluster - print the number of lines (`count()`) of your cleaned RDD from the big file, here:** 

original: 173,179,771
cleaned: 173,176,128

**Repeat task 2 on the big file in your EMR cluster - copy your result, which is the 5 dates with least total revenue, from the big file, here:** 

[('2013-08-11', 3071194.3099999935), 
('2013-08-04', 3073608.610000252), 
('2013-08-03', 3182263.760000205), 
('2013-12-25', 3465875.3099999377), 
('2013-08-02', 3530800.6300003207)]

**Repeat task 3 on the big file in your EMR cluster - copy your result, which is the top 5 drivers with the most revenue, from the big file, here:**  

[('664927CDE376A32789BA48BF55DFB7E3', 728594.33), 
('CFCD208495D565EF66E7DFF9F98764DA', 615220.1499999997), 
('E4F99C9ABE9861F18BCD38BC63D007A9', 563445.9400000001), 
('D85749E8852FCC66A990E40605607B2F', 246374.5899999999), 
('1EDF99EE9DAC182027330EF48828B54A', 242656.10000000003)]   

**Repeat task 4 on the big file in your EMR cluster. `2013_weekdays.csv` is also available in S3 through this URI `s3://comp643bucket/homework/spark_taxicab/2013_weekdays.csv`. Copy your result, which is the sum of revenue per weekday in descending order of total revenue, from the big file, here:**  

[('Friday', 394663376.4300212), 
('Thursday', 386085572.97001725), 
('Wednesday', 372733935.33001846), 
('Saturday', 368934554.230014), 
('Tuesday', 362666230.2000157), 
('Sunday', 341485998.91001475), 
('Monday', 334818004.1200151)]                